In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer,Input, Embedding, LSTM, GRU, Conv1D, Conv2D, GlobalMaxPool1D, Dense, Dropout,Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
import tensorflow.keras as keras
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim
import gensim.downloader
import numpy as np
import itertools
from numpy.linalg import norm
import pandas as pd
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def preprocess_data(train, test):
    for dataset in [train, test]:
        for i, row in dataset.iterrows():
            if row['question1']:
                yield gensim.utils.simple_preprocess(row['question1'])
            if row['question2']:
                yield gensim.utils.simple_preprocess(row['question2'])

In [4]:
def split_string(string_data): 
    return str(string_data).lower().split()

In [5]:
def create_word_2_vec_model(train, test):
  doc = list(preprocess_data(train, test))
  model = gensim.models.Word2Vec(doc, size=300)
  model.train(doc, total_examples=len(doc), epochs=15)
  return model

In [19]:
def emmbed_data(word2vec_model , dataset, embedding_dim=300 , pretrained = False ):
    vocabs = {}
    index_of_vocab = 0

    if pretrained:
      model = gensim.downloader.load('word2vec-google-news-300')
    else:
      model = word2vec_model.wv

    for index, row in dataset.iterrows():
        for question in ['question1', 'question2']:
            ids_vector = []  
            for word in split_string(row[question]):
                if word in set(stopwords.words('english')):
                    continue
                if word in vocabs:
                  ids_vector.append(vocabs[word])
                else:
                  index_of_vocab += 1
                  vocabs[word] = index_of_vocab
                  ids_vector.append(index_of_vocab)
            dataset.at[index, question] = ids_vector
    embeddings = 1 * np.random.randn(len(vocabs) + 1, embedding_dim)
    embeddings[0] = 0 
    for word, index in vocabs.items():
        if word in model.vocab:
            embeddings[index] = model.word_vec(word)
    return dataset, embeddings

In [7]:
def padding_data(df, max_seq_length):
    X = {'left': df['question1'], 'right': df['question2']}
    for dataset, side in itertools.product([X], ['left', 'right']):
        dataset[side] = pad_sequences(dataset[side], padding='pre', truncating='post', maxlen=max_seq_length)
    return dataset

In [8]:
def create_model(n_hidden,embeddings,embedding_dim,max_seq_length, is_bidirectional = False):
  s = Sequential()
  s.add(Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_shape=(max_seq_length,), trainable=False))
  if is_bidirectional:
    s.add(Bidirectional(LSTM(n_hidden)))
  else:
    s.add(LSTM(n_hidden))
  l_in = Input(shape=(max_seq_length,), dtype='int32')
  r_in = Input(shape=(max_seq_length,), dtype='int32')
  distance =keras.layers.Dot(axes=(1), normalize=True)([s(l_in), s(r_in)])
  model = Model(inputs=[l_in, r_in], outputs=[distance])
  return model

In [9]:
def plot_results(hist):
  plt.subplot(211)
  plt.plot(hist.history['accuracy'])
  plt.plot(hist.history['val_accuracy'])
  plt.title(' Accuracy lstm +pretrained word2vec')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  
  plt.subplot(212)
  plt.plot(hist.history['loss'])
  plt.plot(hist.history['val_loss'])
  plt.title('Loss lstm +pretrained word2vec')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper right')
  plt.show()

In [10]:
def save_array(address , array):
  with open(address, 'wb') as f:
    np.save(f, array,allow_pickle=True)

In [11]:
def predict_usnig_model(model,test_df,embedding_dim,max_seq_length):
  test_df, embeddings = emmbed_data(test_df, embedding_dim=embedding_dim, empty_w2v=not use_w2v)
  results = []
  test_df_unique = test_df.drop_duplicates(subset = 'qid1')
  test_df_unique = test_df_unique.reset_index()
  for i in range(len(test_df_unique)):
    test_data = test_df_unique.iloc[i]
    list_of_pairs = []
    df_pairs = train_df[["qid2","question2"]].drop_duplicates(subset = 'qid2')
    df_pairs = df_pairs.reset_index()
    df_pairs["qid1"] = test_data["qid1"]
    df_pairs["question1"]="t"
    for index in df_pairs.index:
      df_pairs.at[index, "question1"] = test_data["question1"]
    df_pairs_2 = padding_data(df_pairs, max_seq_length)
    prediction = model.predict([df_pairs_2['left'], df_pairs_2['right']])
    df_pairs["scores"] = prediction
    sort = df_pairs.sort_values(by=['scores'],ascending = False)
    res = [sort.iloc[0]["qid1"],np.asarray(sort.iloc[:10]["qid2"])]
    results.append(res)
  return results

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [16]:
train_Address = '/content/drive/MyDrive/IR/HW2/train_data.csv'
test_Address = '/content/drive/MyDrive/IR/HW2/test_data.csv'
validation_Address = '/content/drive/MyDrive/IR/HW2/valid_data.csv'

embedding_dim = 300
max_seq_length = 60
batch_size = 256
n_epoch = 15
n_hidden = 50

In [21]:
train_df = pd.read_csv(train_Address)
test_df = pd.read_csv(test_Address)
valid_df = pd.read_csv(validation_Address)

In [22]:
word_2_vec_model = create_word_2_vec_model(train_df, test_df)
train_df, embeddings = emmbed_data(word_2_vec_model,train_df, embedding_dim=embedding_dim, pretrained = False)
valid_df, embeddings_valid = emmbed_data(word_2_vec_model,valid_df, embedding_dim=embedding_dim, pretrained = False)

X_train = train_df[['question1', 'question2']]
Y_train = train_df['is_duplicate']

X_validation = valid_df[['question1', 'question2']]
Y_validation = valid_df['is_duplicate']

X_train = padding_data(X_train, max_seq_length)
X_validation = padding_data(X_validation, max_seq_length)

Y_train = Y_train.values
Y_validation = Y_validation.values

In [ ]:
model = create_model(n_hidden,embeddings,embedding_dim,max_seq_length, is_bidirectional = False)
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
hist = model.fit([X_train['left'], X_train['right']], Y_train,batch_size=batch_size, epochs=n_epoch,validation_data=([X_validation['left'], X_validation['right']], Y_validation))


In [ ]:
plot_results(hist)

In [ ]:
train_df_predict = pd.read_csv("/content/drive/MyDrive/IR/HW2/train_data.csv")
test_df_predict = pd.read_csv("/content/drive/MyDrive/IR/HW2/test_data.csv")

train_df_predict, _ = emmbed_data(train_df_predict, embedding_dim=embedding_dim, empty_w2v=not use_w2v)
test_df_predict, _ = emmbed_data(test_df_predict, embedding_dim=embedding_dim, empty_w2v=not use_w2v)

X_train_predict = padding_data(train_df_predict, max_seq_length)
X_test_predict = padding_data(test_df_predict, max_seq_length)

In [ ]:
train_dataset_final = train_df_predict[["qid2"]]
train_dataset_final["tensor_2"]="t"
tensor2 = x(X_train_predict['right']) 
for index in range(len(train_dataset_final)):
  train_dataset_final.at[index, "tensor_2"] = tensor2[index].numpy()

test_dataset_final = test_df_predict[["qid1"]]
test_dataset_final["tensor_1"]="t"
tensor1 = x(X_test_predict['left']) 
for index in range(len(test_dataset_final)):
  test_dataset_final.at[index, "tensor_1"] = tensor1[index].numpy()

train_dataset_final = train_dataset_final.drop_duplicates(subset = 'qid2').reset_index()
test_dataset_final = test_dataset_final.drop_duplicates(subset = 'qid1').reset_index()

In [ ]:
results = []
for i in range(len(test_dataset_final)):
  test_data = test_dataset_final.iloc[i]
  df_pairs = train_dataset_final.copy()
  df_pairs["qid1"] = test_data["qid1"]
  df_pairs["tensor_1"]="t"
  df_pairs["prediction"] = "6"
  for index in df_pairs.index:
    tens1 = test_data["tensor_1"]
    tens2 = df_pairs.iloc[index]["tensor_2"]
    df_pairs.at[index, "tensor_1"] = tens1
    dd = np.dot(tens1,tens2)/(norm(tens1)*norm(tens2))
    df_pairs.at[index, "prediction"] = dd
  sort = df_pairs.sort_values(by=['prediction'],ascending = False)
  res = [sort.iloc[0]["qid1"],np.asarray(sort.iloc[:10]["qid2"])]
  results.append(res)

In [ ]:
results_save_address = '/content/drive/MyDrive/IR/HW2/results-semance-lstm-cosine.npy'
save_array(results_save_address , results)

In [ ]:
# predict using model predict
test_df = pd.read_csv("/content/drive/MyDrive/IR/HW2/test_data.csv")
results = predict_usnig_model(model,test_df,300,60)